# Accidents and Lethal Accidents - Geographical Analysis
Ori Moisis, Tal Peleg, Aviram Stern, Noam Hershtig & Moran Neuhof

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Accidents-and-Lethal-Accidents---Geographical-Analysis" data-toc-modified-id="Accidents-and-Lethal-Accidents---Geographical-Analysis-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Accidents and Lethal Accidents - Geographical Analysis</a></span><ul class="toc-item"><li><span><a href="#Looking-at-lethal-accidents" data-toc-modified-id="Looking-at-lethal-accidents-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Looking at lethal accidents</a></span><ul class="toc-item"><li><span><a href="#Accidents-geographical-distribution---by-year" data-toc-modified-id="Accidents-geographical-distribution---by-year-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Accidents geographical distribution - by year</a></span></li><li><span><a href="#Accidents-geographical-distribution---by-hour" data-toc-modified-id="Accidents-geographical-distribution---by-hour-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Accidents geographical distribution - by hour</a></span></li></ul></li><li><span><a href="#Looking-at-all-accidents" data-toc-modified-id="Looking-at-all-accidents-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Looking at all accidents</a></span><ul class="toc-item"><li><span><a href="#Accidents-geographical-distribution---by-year" data-toc-modified-id="Accidents-geographical-distribution---by-year-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Accidents geographical distribution - by year</a></span></li><li><span><a href="#Accidents-geographical-distribution---by-hour" data-toc-modified-id="Accidents-geographical-distribution---by-hour-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Accidents geographical distribution - by hour</a></span></li></ul></li><li><span><a href="#Binned-coordinates" data-toc-modified-id="Binned-coordinates-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Binned coordinates</a></span><ul class="toc-item"><li><span><a href="#Binning-coordinated" data-toc-modified-id="Binning-coordinated-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Binning coordinated</a></span></li><li><span><a href="#Accidents-geographical-distribution---by-year-(binned)" data-toc-modified-id="Accidents-geographical-distribution---by-year-(binned)-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Accidents geographical distribution - by year (binned)</a></span></li><li><span><a href="#Accidents-geographical-distribution---by-hour-(binned)" data-toc-modified-id="Accidents-geographical-distribution---by-hour-(binned)-1.3.3"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>Accidents geographical distribution - by hour (binned)</a></span></li></ul></li></ul></li></ul></div>

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import geopandas
from shapely.geometry import Point

%matplotlib inline

In [ ]:
# Load file
df = pd.read_csv('anyway_tables_csv_updated/involved_markers_hebrew.csv')

In [ ]:
## Preparing coordinates in dataframe
df['Coordinates'] = list(zip(df.longitude, df.latitude))
df['Coordinates'] = df['Coordinates'].apply(Point)
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

## Looking at lethal accidents

In [ ]:
# Subsetting only lethal accidents
lethal_df = df[df['accident_severity']==1].reset_index().drop_duplicates('provider_and_id').reset_index()
lethal_gdf = geopandas.GeoDataFrame(lethal_df, geometry='Coordinates')

### Accidents geographical distribution - by year

In [ ]:
for i, year in enumerate(range(2008, 2019)):
    ax = world[world.name == 'Israel'].plot(color='white', edgecolor='black', figsize=(16,9))
    lethal_df_year = lethal_gdf[lethal_gdf['accident_year'] == year]
    lethal_df_year.plot(ax=ax, color='red',alpha=0.3)
    plt.title(year)
    plt.savefig(f"{year}.png")

### Accidents geographical distribution - by hour

In [ ]:
for i, hour in enumerate(range(0, 24)):
    ax = world[world.name == 'Israel'].plot(color='white', edgecolor='black', figsize=(16,9))
    lethal_gdf_hour = lethal_gdf[lethal_gdf['accident_hour'] == hour]
    lethal_gdf_hour.plot(ax=ax, color='red',alpha=0.3)
    plt.title(hour)
    plt.savefig(f"hour_of_day{hour}.png")

## Looking at all accidents

In [ ]:
gdf = geopandas.GeoDataFrame(df, geometry='Coordinates')

### Accidents geographical distribution - by year

In [ ]:
for i, year in enumerate(range(2008, 2019)):
    ax = world[world.name == 'Israel'].plot(color='white', edgecolor='black', figsize=(16,9))
    gdf_year = gdf[gdf['accident_year'] == year]
    gdf_year.plot(ax=ax, color='red',alpha=0.3)
    plt.title(year)
    plt.savefig(f"all_accidents_year_{year}.png")

### Accidents geographical distribution - by hour

In [ ]:
for i, hour in enumerate(range(0, 24)):
    ax = world[world.name == 'Israel'].plot(color='white', edgecolor='black', figsize=(16,9))
    gdf_hour = gdf[gdf['accident_hour'] == hour]
    gdf_hour.plot(ax=ax, color='red',alpha=0.3)
    plt.title(hour)
    plt.savefig(f"all_accidents_hour_of_day{hour}.png")

## Binned coordinates

### Binning coordinated

In [ ]:
# Binned distributions
# Accurate DF (with accuracy filter)
accurate_df = df[df['location_accuracy'] == 1].reset_index().drop_duplicates('provider_and_id').reset_index()

lat_cut = pd.cut(accurate_df.latitude, np.linspace(min(accurate_df.latitude), max(accurate_df.latitude), 30), right=True).apply(lambda x: x.mid)
lon_cut = pd.cut(accurate_df.longitude, np.linspace(min(accurate_df.longitude), max(accurate_df.longitude), 20), right=True).apply(lambda x: x.mid)

accurate_df['lon_cut'] = lon_cut
accurate_df['lat_cut'] = lat_cut

accurate_df['Binned_Coordinates'] = list(zip(accurate_df.lon_cut, accurate_df.lat_cut))
accurate_df['Binned_Coordinates'] = accurate_df['Binned_Coordinates'].apply(Point)
# Creating GDF
accurate_gdf = geopandas.GeoDataFrame(accurate_df, geometry='Binned_Coordinates')

### Accidents geographical distribution - by year (binned)

In [ ]:
# Binned
for i, year in enumerate(range(2008, 2019)):
    ax = world[world.name == 'Israel'].plot(color='white', edgecolor='black', figsize=(16,9))
    gdf_year = accurate_gdf[accurate_gdf['accident_year'] == year]
    gdf_year.plot(ax=ax, color='red',alpha=0.3)
    plt.title(year)
    plt.savefig(f"all_accidents_year_{year}.png")

### Accidents geographical distribution - by hour (binned)

In [ ]:
for i, hour in enumerate(range(0, 24)):
    ax = world[world.name == 'Israel'].plot(color='white', edgecolor='black', figsize=(16,9))
    gdf_hour = accurate_gdf[accurate_gdf['accident_hour'] == hour]
    gdf_hour.plot(ax=ax, color='red',alpha=0.3)
    plt.title(hour)
    plt.savefig(f"all_accidents_hour_of_day{hour}.png")

The same can be done for lethal accidents only (by repeating the filter in 1.1)